In [13]:
import numpy as np
from tqdm.notebook import tqdm

## Part 1: Efficient mean & variance calculation

In [35]:
PATH_TO_FILE = './measurement_data_uint8.bin'

with open(PATH_TO_FILE, mode='rb') as file:
    data = file.read()

a. Naive approach

In [23]:
sumData = 0
for i in tqdm(data):
    sumData += i
naiveMean = sumData / len(data)
print(naiveMean)

  0%|          | 0/100000000 [00:00<?, ?it/s]

133.70021891


In [27]:
sumDiffSquare = 0
for i in tqdm(data):
    sumDiffSquare += (i - naiveMean)**2
naiveVariance = sumDiffSquare / len(data)
print(naiveVariance)

  0%|          | 0/100000000 [00:00<?, ?it/s]

1.8707706471619652


Side-notes: working with numpy and int array

In [29]:
intData = np.zeros(len(data))
for i in tqdm(range(0,len(data))):
    intData[i] = data[i]

  0%|          | 0/100000000 [00:00<?, ?it/s]

In [33]:
numpyMean = np.mean(intData)
print(numpyMean)

133.70021891


In [34]:
numpyVariance = np.var(intData)
print(numpyVariance)

1.87077064807842


b. Welford's algorithm 

In [ ]:

def WelfordUpdater(count, mean, M2, newValue):
    (count, mean, M2) = existingAggregate
    count += 1
    delta = newValue - mean
    mean += delta / count
    delta2 = newValue - mean
    M2 += delta * delta2
    return (count, mean, M2)

# Retrieve the mean, variance and sample variance from an aggregate
def finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    if count < 2:
        return float("nan")
    else:
        (mean, variance, sampleVariance) = (mean, M2 / count, M2 / (count - 1))
        return (mean, variance, sampleVariance)